## ChatBot

In [26]:
import sys
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import random
import requests
import gmaps
from gmaps import Geocoding
from random import randint

In [2]:
print("Welcome to chatbot\nTo exit press \"exit\"")

Welcome to chatbot
To exit press "exit"


In [3]:
from nltk.wsd import lesk
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
import tkinter
from nltk.tag import brill, brill_trainer
from nltk.tag.brill_trainer import BrillTaggerTrainer
from nltk.data import load


from nltk.corpus.reader import TaggedCorpusReader
train_data = TaggedCorpusReader('.', 'tagged_input_sentences.txt', sep="/")

traindata= list(train_data.tagged_sents())

postag= load('taggers/maxent_treebank_pos_tagger/english.pickle')

templates = [
        brill.Template(brill.Pos([-1])),
        brill.Template(brill.Pos([1])),
        brill.Template(brill.Pos([-2])),
        brill.Template(brill.Pos([2])),
        brill.Template(brill.Pos([-2, -1])),
        brill.Template(brill.Pos([1, 2])),
        brill.Template(brill.Pos([-3, -2, -1])),
        brill.Template(brill.Pos([1, 2, 3])),
        brill.Template(brill.Pos([-1]), brill.Pos([1])),
        brill.Template(brill.Word([-1])),
        brill.Template(brill.Word([1])),
        brill.Template(brill.Word([-2])),
        brill.Template(brill.Word([2])),
        brill.Template(brill.Word([-2, -1])),
        brill.Template(brill.Word([1, 2])),
        brill.Template(brill.Word([-3, -2, -1])),
        brill.Template(brill.Word([1, 2, 3])),
        brill.Template(brill.Word([-1]), brill.Word([1]))]
        
trainer = BrillTaggerTrainer(postag, templates = templates, trace = 3)
brill_tagger = trainer.train(traindata, max_rules = 10)

TBL train (fast) (seqs: 38; tokens: 331; tpls: 18; min score: 2; min acc: None)
Finding initial useful rules...
    Found 661 useful rules.

           B      |
   S   F   r   O  |        Score = Fixed - Broken
   c   i   o   t  |  R     Fixed = num tags changed incorrect -> correct
   o   x   k   h  |  u     Broken = num tags changed correct -> incorrect
   r   e   e   e  |  l     Other = num tags changed incorrect -> incorrect
   e   d   n   r  |  e
------------------+-------------------------------------------------------
  22  28   6   2  | VB->NNP if Pos:TO@[-1]
   8   8   0   0  | IN->NN if Word:hsr@[-1]
   6   8   2   1  | JJ->NN if Pos:IN@[-3,-2,-1]
   5   5   0   0  | PRP->NN if Pos:VBP@[1]
   2   2   0   0  | VBD->NN if Pos:NN@[-1]
   2   2   0   0  | VBN->JJ if Pos:DT@[-1]
   2   2   0   0  | JJ->NNP if Pos:IN@[1]
   2   2   0   0  | NNP->NN if Pos:DT@[1]
   2   2   0   0  | NNP->VB if Pos:TO@[1]
   2   2   0   0  | VBP->VB if Pos:VB@[2]


In [4]:
def find_intent(inp):
    intent = 'unknown' 
    ### we can try looking for lammatized word and synonyms in the inp rather directly
    ### match patern to get the intent
    if 'exit' in inp:
        intent = 'exit'
    
    '''if inp == 'OK':
        intent = 'final'
    '''
    if 'hey' in inp or 'hello' in inp or 'hi' in inp:
        intent = 'greeting'
        
    if 'book' in inp or 'get' in inp:
        intent = 'booking'
    
    elif 'available' in inp or 'look' in inp:
        intent = 'looking'
    
    return intent
    
### book, look,     (?<!born\s)\d{4}

In [5]:
def extract_location(inp):
    tagged = brill_tagger.tag(word_tokenize(inp))
    print(tagged)
    source = None
    destination = None
    chunkGram = """Source: {<IN>(<NN.*><,>?)+}"""
    chunkParser = nltk.RegexpParser(chunkGram)
    chunked = chunkParser.parse(tagged)
    for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Source'):
        source = ' '.join(list(zip(*subtree))[0][1:]) 
    
    chunkGram = """Destination: {<TO>(<NN.*><,>?)+}"""
    chunkParser = nltk.RegexpParser(chunkGram)
    chunked = chunkParser.parse(tagged)
    for subtree in chunked.subtrees(filter=lambda t: t.label() == 'Destination'):
        destination = ' '.join(list(zip(*subtree))[0][1:]) 
    #print('function end',source,destination)
    return source, destination

In [6]:
def handle_require_field(intent):
    ### keys needed for booking 
    if intent== 'booking': requiredKey = ['source','destination']
    elif intent == 'looking' : requiredKey = ['source']
    ### storing missing key require to complete booking
    notAvail = []
    for key in requiredKey:
        if key not in SessInfo:
            notAvail.append(key)
        ### we can set flag and ask for all the keys simuntaneously
    if(len(notAvail)!=0):
        print('Please provide the following details to proceed:\n')
        for key in notAvail:
            print(key)
            SessInfo[key] = input()    

In [ ]:
def storing_info(inp, intent, SessInfo):
## intent might just be needed for output generation
## removed Intent   
    #print(inp, intent)
    if intent == 'greeting':
        #pattern = re.compile(r'\bHi|Hello|Hey)\b')
        match = re.search('\\bhi|hello|hey\\b', inp)
        if match : SessInfo['greet'] = match.group()
        
    if intent == 'booking' or intent == 'looking':
    ### look for name entity and of 2 location are ther ethen look for from and 
    ### to and just store the one tagged with gpe
        
      ##########################################################
    ############################################################
    '''        if 'from' in inp and 'to' in inp:
            print('***Storing from and to')
            match = re.search('(?<=from\s)\w+\\b', inp)
            if match:
                SessInfo['source'] = match.group()

            match = re.search('(?<=to\s)\w+\\b', inp)
            if match:
                SessInfo['destination'] = match.group()
            #return
        elif 'from' in inp:
            print('***Storing from')
            match = re.search('(?<=from\s)\w+\\b', inp)
            if match:
                SessInfo['source'] = match.group()
                if 'destination' in SessInfo: del SessInfo['destination']
            #return
        elif 'to' in inp:
            print('***Storing to')
            match = re.search('(?<=to\s)\w+\\b', inp)
            if match:
                SessInfo['destination'] = match.group()
                if 'source' in SessInfo: del SessInfo['source']'''
    ###########################################################
    ################################################################
        
        source , destination = extract_location(inp)
        #print(source, destination)
        
        if source and destination:
            #print('both')
            SessInfo['source'] = source
            SessInfo['destination'] = destination
        elif destination:
            #print('matched field is destination')
            SessInfo['destination'] = destination
        elif source:
            #print('matched field is source')
            SessInfo['source'] = source

In [8]:
destination = None
if destination : print(destination)

In [19]:
def cab_details(ride):
    columns = ['Car type', 'Availability', 'Number of seats', 'Distance(Km)', 'Estimated Travel time(min)', 'Minimum Price', 'Maximum Price','ETA' ]
    #df = pd.DataFrame(data,columns = columns)
    ride_cat = ride['categories']
    ride_est = ride['ride_estimate']
    dict = []
    for i in range(0,len(ride_cat)):
        if ride_cat[i]['eta'] == 0:
            print (ride_cat[i]['display_name'] + " not available")
            data1 = {'Car type' : ride_cat[i]['display_name'], 'Availability': "No"}
            dict.append(data1)
        else:
            avail='Yes'
            if ride_cat[i]['eta'] == -1: avail='No'
            if ride_cat[i]['display_name'] == "Share":
                for j in range(0,2):
                    data1 = {'Car type': ride_cat[i]['display_name'],
                             'Number of seats': ride_est[i]['fares'][j]['seats'], 
                             'Travel time(min)': ride_est[i]['travel_time_min'], 
                             'Maximum Price': ride_est[i]['fares'][j]['cost'], 
                             'Minimum Price': ride_est[i]['fares'][j]['cost'], 
                             'Availability' : avail,
                             'ETA': ride_cat[i]['eta']}
                    dict.append(data1)
            else:
                data1 = {'Car type': ride_cat[i]['display_name'],
                         'Number of seats': "4",
                         'Distance(Km)': ride_est[i]['distance'], 
                         'Estimated Travel time(min)': ride_est[i]['travel_time_in_minutes'], 
                         'Maximum Price': ride_est[i]['amount_max'], 
                         'Minimum Price': ride_est[i]['amount_min'], 
                         'Availability' : avail,
                         'ETA': ride_cat[i]['eta']}
                dict.append(data1)          
    df = pd.DataFrame(dict,columns = columns)
    return df

In [31]:
def cab_details_src(ride):
    columns = ['Car type', 'Availability', 'Number of seats', 'ETA', 'Distance' ]
    ride_cat = ride['categories']
    dicti = []
    for i in range(0,len(ride_cat)):
        seats = "4"
        avail = "Yes"
        if ride_cat[i]['eta'] == -1:
            avail = "No"
            data = {'Car type' : ride_cat[i]['display_name'],'Availability' : avail}
            dicti.append(data)
        else:
            if ride_cat[i]['display_name']=="Share":
                seats = "1/2"
                data = {'Car type' : ride_cat[i]['display_name'],'Availability' : avail, 'Number of seats':           seats,'ETA' : ride_cat[i]['eta'],'Distance' : ride_cat[i]['distance']}
                dicti.append(data)   
            else:
                data = {'Car type' : ride_cat[i]['display_name'],'Availability' : avail, 'Number of seats': seats,'ETA' : ride_cat[i]['eta'],'Distance' : ride_cat[i]['distance']}
                dicti.append(data)
    df = pd.DataFrame(dicti,columns = columns)
    return df

In [32]:
def look(source, destination=None):
    api = Geocoding()
    source_loc = api.geocode(source)
    source_lat_long = source_loc[0]['geometry']['location']
    if destination : 
        destination_loc = api.geocode(destination)
        destination_lat_long = destination_loc[0]['geometry']['location']
        payload = {'pickup_lat': str(source_lat_long['lat']), 'pickup_lng':str(source_lat_long['lng']), 'drop_lat':str(destination_lat_long['lat']), 'drop_lng':str(destination_lat_long['lng']), 'pickup_time': '19/4/2017 16:30'}
    else:
        #print('am here')
        payload = {'pickup_lat': str(source_lat_long['lat']), 'pickup_lng':str(source_lat_long['lng']), 'pickup_time': '19/4/2017 16:30'}
    
    headers = {'X-APP-TOKEN' : "ab8a1c87e9f64ba3a36c2858ad9c08c2"}
    response = requests.get('http://sandbox-t.olacabs.com/v1/products', params=payload, headers=headers)
    if destination : cab_details_df = cab_details(response.json())
    else: cab_details_df = cab_details_avai(response.json())
    cab_details_df.fillna(0,axis=1,inplace=True)
    return cab_details_df
    #### response is the json reply we get 

In [33]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

In [34]:
def book(source, destination, cabtype, noofseats):
    df = look(source, destination)
    driver_name=["Tharoon Veerasethu","Sharmaji","Chachaji","Chunnu","Munni","beta","daadaji","daadi","bhai","shinde","langoor"]
    for i in range(df.shape[0]):
        x=list(df.loc[i])
        if cabtype in x:
            for j in x:
                if j == cabtype and df.loc[i,'Availability']=='Yes':
                    SessInfo['bookingid']="OLA"+str(random_with_N_digits(6))
                    SessInfo['drivername']=random.choice(driver_name)
                    SessInfo['driverno']=random_with_N_digits(10)
                    SessInfo['carno']="KN-"+str(random_with_N_digits(2))+"-"+str(random_with_N_digits(4))
                    SessInfo['fare']=(df.loc[i,'Minimum Price']+df.loc[i,'Maximum Price'])/2
                    SessInfo['cabtype']=cabtype
                    SessInfo['noofseats']=noofseats
                    SessInfo['eta']=df.loc[i,'ETA']
                    if cabtype != 'Share': 
                        SessInfo['distance']=df.loc[i,'Distance(Km)']
                        Sessinfo['ett']=df.loc[i,'Estimated Travel time(min)']
                    break                   

In [35]:
def generating_response(inp, intent, SessInfo):
    if intent == 'exit':
        sys.exit()
    else:
        if intent == 'greeting':
            greet = ['Hey', 'Hello', 'Namaste', 'Hi']
            print(random.choice(greet), ' to you too.')

        ### For handling response to booking queries
        
        if intent == 'looking' :
            handle_require_field(intent)
                
            print('Looking for a cab from {} {}'.format(SessInfo.get('source',''),'to '+SessInfo['destination'] if 'destination' in SessInfo else '' ))
            print('Firing up cab looking. Please Wait\n')
            ## call look function (source to destination#optional)
            response_avail_df = look(SessInfo.get('source',''),SessInfo.get('destination',None))    
            #print('will wait for succesful query and cabs info')
            #query ='sucessful'
            #if query == 'sucessful':
            print('Following are the cabs availability\n')
            print(response_avail_df)
            print('Would you like to book any cab. If yes please reply with YES')
            reply = input()
            if reply == 'yes':
                handle_require_field('booking')
                #### book function
                print('Please provide the cab type')
                cab_type_input = input()                    
                available_cab_category = ['micro','mini','sedan']
                if cab_type_input in available_cab_category:
                    ## fire cab booking with 

                    pass
                else:
                    print('Sorry {} not available. Please provide correct response'.format(cab_type_input))
                ######## book function   
            else:
                print('Ok.\n Let\'s start again.')
                if 'source' in SessInfo:del SessInfo['source']
                if 'destination' in SessInfo:del SessInfo['destination']
                return
        
        elif intent == 'booking' :
            
            '''for key in SessInfo:
                print(key, SessInfo[key])'''
            
            handle_require_field(intent)                
                        
            print('Please confirm witk \'OK\' to book a cab from {} to {}'.format(SessInfo['source'],SessInfo['destination']))
            confirm_inp = input()
            if confirm_inp.lower() == 'ok':
                ## call look function (source to destination)
                ## show available cabs and ask user input to book any cab
                print('Firing up cab booking. Please Wait')
                print('will wait for booing id, update in sessInfo and return then, else return with error') 
            else:
                if 'source' in SessInfo:del SessInfo['source']
                if 'destination' in SessInfo:del SessInfo['destination']
                print('Let\'s start the conversation again')
                return            
                
            

In [ ]:
### storing important info
SessInfo = dict() ## refreesh dictionary when user work is completed
while(True):
    ## letting the case as it is
    inp = input().lower()
    
    ### find the intent
    intent = find_intent(inp)
    
    
    ### look for a particular patterns as per intent
        ### update the dict
    storing_info(inp, intent, SessInfo)
    
    
    '''for key in SessInfo:
        print(key, SessInfo[key])'''

    ### generate output
    #### function1 (intent, dict) ## intent is greeting reply normally
    generating_response(inp, intent, SessInfo)

look for a cab from kr puram
[('look', 'NN'), ('for', 'IN'), ('a', 'DT'), ('cab', 'NN'), ('from', 'IN'), ('kr', 'NN'), ('puram', 'NN')]
Looking for a cab from kr puram 
Firing up cab looking. Please Wait

Following are the cabs availability

      Car type Availability  Number of seats  ETA  Distance
0        Micro           No              0.0  0.0       0.0
1         Mini           No              0.0  0.0       0.0
2        Sedan           No              0.0  0.0       0.0
3  Prime Sedan           No              0.0  0.0       0.0
4        Share           No              0.0  0.0       0.0
Would you like to book any cab. If yes please reply with YES
yes
Please provide the following details to proceed:

destination
wipro, electronic city
Please provide the cab type
mini


In [ ]:
def look(source,destination):
    api = Geocoding()
    source_loc = api.geocode(source)
    destination_loc = api.geocode(destination)
    source_lat_long = source_loc[0]['geometry']['location']
    destination_lat_long = destination_loc[0]['geometry']['location']
    headers = {'X-APP-TOKEN' : "ab8a1c87e9f64ba3a36c2858ad9c08c2"}
    payload= {'pickup_lat': str(source_lat_long['lat']), 'pickup_lng':str(source_lat_long['lng']), 'drop_lat':str(destination_lat_long['lat']), 'drop_lng':str(destination_lat_long['lng']), 'pickup_time': '19/4/2017 16:30'}
    response = requests.get('http://sandbox-t.olacabs.com/v1/products', params=payload, headers=headers)
    #cab_details_df=cab_details(response.json())
    #return cab_details_df 
    print(response.json())

### Analysis using the Sample Input Sentences

In [161]:
# Reading all the candidate input sentences
with open('sample-input-sentences.txt') as f:
        train_data = f.read()

In [162]:
# Training sentences
train_sentences = train_data.strip().split('\n')

In [163]:
def get_ne_chunks(sentence):
    tokenized_words = word_tokenize(sentence)
    tagged_words = nltk.pos_tag(tokenized_words)
    ne_chunks = nltk.ne_chunk(tagged_words)
    return ne_chunks

In [176]:
# Applying the default NE chunker over few input sentences
[extract_location(sentence) for sentence in train_sentences[0:5]]

[('Book', 'NNP'), ('the', 'DT'), ('cab', 'NN'), ('from', 'IN'), ('Kormangla', 'NNP'), ('to', 'TO'), ('Whitefield', 'NNP')]
[('Book', 'NNP'), ('the', 'DT'), ('cab', 'NN'), ('from', 'IN'), ('Sarjapur', 'NNP'), ('to', 'TO'), ('Bellandur', 'NNP')]
[('Book', 'NNP'), ('a', 'DT'), ('cab', 'NN'), ('from', 'IN'), ('Shollingnallur', 'NNP'), ('to', 'TO'), ('Sowcarpet', 'NNP')]
[('Boook', 'NNP'), ('a', 'DT'), ('cab', 'NN'), ('from', 'IN'), ('Sowcarpet', 'NNP'), ('to', 'TO'), ('Vepery', 'NNP')]
[('Book', 'NNP'), ('a', 'DT'), ('cab', 'NN'), ('to', 'TO'), ('Marathalli', 'NNP')]


[('Kormangla', 'Whitefield'),
 ('Sarjapur', 'Bellandur'),
 ('Shollingnallur', 'Sowcarpet'),
 ('Sowcarpet', 'Vepery'),
 (None, 'Marathalli')]

In [189]:
for sentence in train_sentences:
    tagged_sentence = []
    for tagged_word in nltk.pos_tag(nltk.word_tokenize(sentence.lower())):
        tagged_sentence.append(nltk.tuple2str(tagged_word))
    print(' '.join(tagged_sentence))

book/NN the/DT cab/NN from/IN kormangla/NN to/TO whitefield/VB
book/NN the/DT cab/NN from/IN sarjapur/NN to/TO bellandur/VB
book/NN a/DT cab/NN from/IN shollingnallur/NN to/TO sowcarpet/VB
boook/VB a/DT cab/NN from/IN sowcarpet/NN to/TO vepery/VB
book/NN a/DT cab/NN to/TO marathalli/VB
book/NN a/DT cab/NN from/IN kormangla/NN to/TO hebbal/VB
can/MD you/PRP please/VB a/DT cab/NN from/IN hebbal/NN to/TO kormangla/VB
book/NN a/DT cab/NN to/TO hsr/VB layout/NN from/IN kormangla/NN
book/NN a/DT cab/NN from/IN hsr/NN layout/NN to/TO kormangla/VB
i/NN want/VBP to/TO book/NN a/DT cab/NN from/IN hsr/NN layout/NN to/TO kormangla/VB
book/NN a/DT cab/NN for/IN ganesh/NN from/IN hsr/NN layout/NN to/TO whitefield/VB
estimate/JJ price/NN from/IN kormangla/NN to/TO whitfield/VB
book/NN a/DT cab/NN to/TO wipro/VB ,/, whitefiled/VBN from/IN teacher/NN 's/POS colony/NN
estimate/VB the/DT price/NN for/IN the/DT travel/NN from/IN whitefiled/VBN to/TO teachers/NNS colony/NN
estimate/NN cost/NN from/IN hebba